In [2]:
import matplotlib.pyplot as plt
import random
from keras.layers import UpSampling2D, concatenate
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras import Model
from numpy import array
from os import listdir
from os.path import join


In [3]:
way_to_original_train = r"E:\folder_J\images\jpg\train\images_jpg"
way_to_edited_train = r"E:\folder_J\images\jpg\train\images_edited_jpg"


def load_images(way: str, lst_of_name: list, flag: bool) -> array:

    images = []
    for name in lst_of_name:
        image = load_img(join(way, name),
                         color_mode="grayscale",
                         target_size=(1150, 180))
        image = img_to_array(image) / 255

        images.append(image)

    return array(images)

In [5]:
input_layer = Input(shape=(1150, 180, 1), batch_size=None)
conv1_1 = Conv2D(8, (3, 3), padding="same", activation="relu")(input_layer)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1_1)

conv2_1 = Conv2D(16, (3, 3), padding="same", activation="relu")(pool1)
pool2 = MaxPooling2D(pool_size=(5, 2))(conv2_1)

conv3_1 = Conv2D(64, (3, 3), padding="same", activation="relu")(pool2)
# pool3 = MaxPooling2D(pool_size=(5, 3))(conv3_2)

conc4 = concatenate([UpSampling2D(size=(5, 2))(conv3_1), conv2_1])
conv4_2 = Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu")(conc4)

conc5 = concatenate([UpSampling2D(size=(2, 2))(conv4_2), conv1_1])
conv5_1 = Conv2D(16, kernel_size=(3, 3), padding="same", activation="relu")(conc5)

conv4 = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(conv5_1)

model = Model(inputs=[input_layer], outputs=[conv4])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[MeanIoU(num_classes=50), "accuracy"])


In [4]:
model.save('model_u.h5')

In [6]:
i, j = 0, 1000
for _ in range(7):
    
    images_name_train = listdir(way_to_original_train)
    images_name_train = sorted(images_name_train, key=lambda *args: random.random())[i:j]

    x_train = load_images(way_to_original_train, images_name_train, False)
    y_train = load_images(way_to_edited_train, images_name_train, True)
    print("finished")
    
    his = model.fit(x_train, y_train, batch_size=50, epochs=2)
    model.save('model_u.h5')
    
    i += 1000
    j += 1000
    if i == 7000:
        j = 7804
    

finished
Epoch 1/2
20/20 [==============================] - 343s 17s/step - loss: 0.2294 - mean_io_u: 0.4984 - accuracy: 0.9786
Epoch 2/2
20/20 [==============================] - 342s 17s/step - loss: 0.0612 - mean_io_u: 0.4984 - accuracy: 0.9789
finished
Epoch 1/2
20/20 [==============================] - 342s 17s/step - loss: 0.0400 - mean_io_u: 0.4984 - accuracy: 0.9796
Epoch 2/2
20/20 [==============================] - 341s 17s/step - loss: 0.0349 - mean_io_u: 0.4984 - accuracy: 0.9796
finished
Epoch 1/2
20/20 [==============================] - 342s 17s/step - loss: 0.0240 - mean_io_u: 0.4984 - accuracy: 0.9794
Epoch 2/2
20/20 [==============================] - 341s 17s/step - loss: 0.0137 - mean_io_u: 0.4984 - accuracy: 0.9802
finished
Epoch 1/2
20/20 [==============================] - 341s 17s/step - loss: 0.0107 - mean_io_u: 0.4985 - accuracy: 0.9811
Epoch 2/2
20/20 [==============================] - 341s 17s/step - loss: 0.0091 - mean_io_u: 0.4985 - accuracy: 0.9813
finished
Epo

In [10]:
import numpy as np
import tensorflow as tf
from PIL import Image
name = listdir(r"E:\folder_J\images\jpg\test\images_for_test_original")
print(name[0])
img = load_img(join(r"E:\folder_J\images\jpg\test\images_for_test_original", name[0]),
                         color_mode="grayscale",
                         target_size=(1150, 180))
img_a = img
img = img_to_array(img) / 255
img = tf.expand_dims(img, axis=0)
res = model.predict(img, batch_size=None)
res = np.reshape(res, (1150, 180))
print(res)
for i in range(len(res)):
    res[i] = np.array(list(map(lambda x: round(x * 255, 0), res[i])))
print(res)
i_res = Image.fromarray(res)
i_res.show()
img_a.show()

1.jpg
[[0.19451198 0.09105033 0.0749231  ... 0.04359952 0.06217757 0.14489797]
 [0.09575582 0.02320141 0.01657838 ... 0.0059275  0.01115516 0.05537364]
 [0.08301824 0.01586789 0.01110893 ... 0.00295013 0.00594661 0.03581381]
 ...
 [0.12822914 0.05856496 0.05416325 ... 0.02144867 0.03111422 0.05828443]
 [0.18936723 0.09707144 0.08368233 ... 0.04096705 0.0566743  0.08943063]
 [0.28587693 0.15641409 0.13536125 ... 0.08926147 0.10327268 0.1569106 ]]
[[50. 23. 19. ... 11. 16. 37.]
 [24.  6.  4. ...  2.  3. 14.]
 [21.  4.  3. ...  1.  2.  9.]
 ...
 [33. 15. 14. ...  5.  8. 15.]
 [48. 25. 21. ... 10. 14. 23.]
 [73. 40. 35. ... 23. 26. 40.]]


In [13]:
model.save('model_u.h5')

In [5]:
from tensorflow import keras
model = keras.models.load_model("model_u.h5")

In [6]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from numpy import array
from os import listdir
from os.path import join

way_to_original_train = r"E:\folder_J\images\jpg\test\images_for_test_original"
way_to_edited_train = r"E:\folder_J\images\jpg\test\images for_test_edited"


def load_images(way: str, lst_of_name: list, flag: bool) -> array:

    images = []
    for name in lst_of_name:
        image = load_img(join(way, name),
                         color_mode="grayscale",
                         target_size=(1150, 180))
        image = img_to_array(image) / 255

        images.append(image)

    return array(images)

In [7]:
x_train = load_images(way_to_original_train, images_name_train, False)
y_train = load_images(way_to_edited_train, images_name_train, True)
print("finished")

NameError: name 'images_name_train' is not defined

In [ ]:
model.evaluate(x_train, y_train)